In [1]:
# Determining Max `number_of_new_images` for Image Augmentation 

In [3]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

utah = pd.read_csv("~/Downloads/UtahHouseScore1.csv")
utah_images = pd.read_csv("~/Downloads/utah_house_score1.csv")
utah.head()
list(utah.columns)
numeric = pd.concat([utah_images.iloc[:99], utah[['sq_ft', 'bedrooms', 'bathrooms', 'patios', 'sq_ft']]], axis=1)
numeric.head()
numeric.to_csv('~/workspace/data-science-scripts/jjohnson/num_cols.csv')

category = pd.concat([utah_images.iloc[:99], utah[['sq_ft', 'school_district', 'elementary', 'jr_high', 'high_school']]], axis=1)
category.to_csv('~/workspace/data-science-scripts/jjohnson/category_cols.csv')

captions = pd.read_csv('~/Downloads/captions.txt')

test_data = []
cols = 250
rows = 305
test_df = pd.DataFrame()
df = pd.DataFrame(np.zeros([rows, cols])*np.nan)

for i in range(rows):
    samples = captions['caption'].sample(cols)
    samples.reset_index(drop=True)
    df.iloc[i] = samples

bigtext = pd.concat([utah_images, df], axis=1)
bigtext.to_csv("~/workspace/data-science-scripts/jjohnson/text_250cols.csv")
#text_captions = pd.DataFrame(test_data)

text = pd.concat([utah_images, utah[['sq_ft', 'description', 'garage_type', 'amenities', 'heating']]], axis=1)
text.to_csv("~/workspace/data-science-scripts/jjohnson/text_cols.csv")

geo = pd.concat([utah_images.iloc[:99], utah[['sq_ft', 'zip_geometry', 'latitude', 'longitude']]], axis=1)
geo['zip_geo2'] = geo['zip_geometry']
geo.to_csv("~/workspace/data-science-scripts/jjohnson/geo_cols.csv")

images = utah_images.copy()
geo = pd.concat([utah_images.iloc[:99], utah[['sq_ft']]], axis=1)
images['img2'] = images['image']
images['img3'] = images['image']
images['img4'] = images['image']
images['img5'] = images['image']
images.to_csv('~/workspace/data-science-scripts/jjohnson/image_cols.csv')

mixed = pd.concat([utah_images.iloc[:99], utah.copy()], axis=1)
mixed.to_csv("~/workspace/data-science-scripts/jjohnson/mixed.csv")

KeyboardInterrupt: 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

#utah = pd.read_csv("~/Downloads/Utah-250.csv")


# Data is leaderboard exports of Imange Tunning Yaml run through shrink.
hundred = pd.read_csv("./100x.csv")
fivehundred = pd.read_csv("500x.csv")
twentyfive = pd.read_csv("25x.csv")
ten = pd.read_csv("10x.csv")
five = pd.read_csv("5x.csv")
mixed_750 = pd.read_csv("~/Downloads/img_aug_mixed_ds_x750_lb.csv")

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 10000)

# some data doesn't have max ram gb calculated
hundred['Max RAM GB'] = hundred['Max_RAM'] / 1000000000
ten['Max RAM GB'] = ten['Max_RAM'] / 1000000000
five['Max RAM GB'] = five['Max_RAM'] / 1000000000
mixed_750['Max RAM GB'] = mixed_750['Max_RAM'] / 1000000000


# combine them all and use total images moving forward
all_data = pd.concat([five, ten, twentyfive, hundred, fivehundred, mixed_750], axis=0)
all_data['total images'].update(all_data.pop('max total images'))

# the assumption is that final task and network type will have a large effect
# so pul those out for graphing purpose
main_tasks =all_data.main_task.unique()
main_tasks_dict = dict( zip(main_tasks, range(1, len(main_tasks) + 1)))
all_data['main_task_int'] = all_data.main_task.apply(lambda x: main_tasks_dict[x])

all_data['network_type'] = all_data.Blueprint.str.extract('net=([^;]*)', expand=False)
all_data['network_type'] = all_data['network_type'].fillna('None')
net_types = all_data['network_type'].unique()
net_types_dict = dict( zip(net_types, range(1, len(net_types) +1)))
all_data['network_type_int'] = all_data.network_type.apply(lambda x: net_types_dict[x])

In [ ]:

# leaderboard export of failed runs won't have MAX RAM GB
valid_ram_data = all_data[all_data['Max RAM GB'] > 0]
valid_ram_data = valid_ram_data.sort_values(by=['Max RAM GB'], ascending=False)

# filter total amount of data, for better visibility (currently to keep graphs as same scale)
#valid_ram_data = valid_ram_data[valid_ram_data['total images'] < 1000000]

# mobilenet shows two distintive trend lines for memory use (graphs below)
# to determine what was different between the two trends I split them so I could let DR
# train for `high_mem_usage` and tell me why the difference
valid_ram_data['images_per_GB'] = valid_ram_data['total images'] / valid_ram_data['Max RAM GB']
valid_ram_data['high_mem_usage'] = valid_ram_data['images_per_GB'] < 11000
valid_ram_data.index = range(len(valid_ram_data))


# I want to use word cloud for the blueprint field, so this string manipulations were done
# iteratively to filter out noise in the word cloud
# remove uuid
valid_ram_data["Blueprint"] = valid_ram_data["Blueprint"].str.replace(r'[0-9a-f]{24}','', regex=True)
# SCPICK ids
valid_ram_data["Blueprint"] = valid_ram_data["Blueprint"].str.replace(r"cn=[0-9a-f]+\'",'', regex=True)
# try to seperate paramets into things DR will tokenize
valid_ram_data["Blueprint"] = valid_ram_data["Blueprint"].str.replace(';',' ')
valid_ram_data["Blueprint"] = valid_ram_data["Blueprint"].str.replace('.', '_')
valid_ram_data["Blueprint"] = valid_ram_data["Blueprint"].str.replace('=','oo')
# remove dense array data for LR1 as it signals LR1 anyways, and this will cleanup the word cloud
valid_ram_data["Blueprint"] = valid_ram_data["Blueprint"].str.replace(r'Coo\[.*\]', 'dense_array_data', regex=True)


# break up the data into networks
squeezenet = 'netoosqueezenet'
efficientnet = 'netooefficientnet'
mobilenet = 'netoomobilenet'

mobilenet_data = valid_ram_data[valid_ram_data['Blueprint'].str.contains(mobilenet)]
efficientnet_data = valid_ram_data[valid_ram_data['Blueprint'].str.contains(efficientnet)]
squeezenet_data = valid_ram_data[valid_ram_data['Blueprint'].str.contains(squeezenet)]



In [ ]:
# export to use in DR
valid_ram_data.to_csv("valid_ram_data.csv")
no_squeeze = valid_ram_data[~valid_ram_data['Blueprint'].str.contains(squeezenet)]
no_squeeze.to_csv("no_squeeze_blueprint_not_id_valid_ram_data.csv")
mobile_net_only = no_squeeze[~no_squeeze['Blueprint'].str.contains(efficientnet)]
mobile_net_only.to_csv("mobilenet_valid_ram_data.csv")
#all_data.to_csv("all_data.csv")

In [ ]:

# graph by network, and split tasks by main_task inorder to find what network / task is the most greedy memory wise
# if we can prevent that combination from OOMing then we will prevent all combinations, this allows us to keep
# the metric (number_of_new_images) project based, otherwise I feel like we would need to do it for each (group) model

# The graphs seemed to show that mobilenet with SGDC is the algorithm to hit 60GB of memory with the least amount of
# images
# graph is Max Ram vs Main Task code, size of dots is the number of images in the dataset.

def plot_data(valid_ram_data, axis, title, color='red'):
    scatter = axis.scatter(valid_ram_data['main_task'], #valid_ram_data['size_factor'], 
            valid_ram_data['Max RAM GB'], 
            c=valid_ram_data['dataset_x_cols'],
            s=valid_ram_data['total images'] / 100,
            alpha=0.5)
    axis.set_title(title)
    # produce a legend with the unique colors from the scatter
    legend1 = axis.legend(*scatter.legend_elements(),
                        loc="lower left", title="# cols")
    axis.add_artist(legend1)

fig, axs = plt.subplots(3)
fig.suptitle("Ram usage per network")
plot_data(squeezenet_data, axs[0], "squeezenet")
plot_data(efficientnet_data, axs[1], "efficientnet")
plot_data(mobilenet_data, axs[2], "mobilenet")



fig = plt.gcf()
fig.set_size_inches(21,18, forward=True)
plt.show()

In [ ]:
# GB vs total images x final modeler x network
from matplotlib import ticker

@ticker.FuncFormatter
def get_task_name(task_int, pos):
    for key, val in main_tasks_dict.items():
        if val == task_int:
            return key
    return task_int

def plot_data(valid_ram_data, axis, title, color='red'):
    scatter = axis.scatter(valid_ram_data['total images'],
            valid_ram_data['Max RAM GB'], 
            #s=valid_ram_data['dataset_x_image'],
            c=valid_ram_data['main_task_int'],
            label=valid_ram_data['main_task'],
            alpha=0.5)
    axis.set_title(title)
    
    legend1 = axis.legend(*scatter.legend_elements(prop='colors', fmt=get_task_name),
                    loc="best", title="Task Codes")
    axis.add_artist(legend1)

    axis.grid(True)
   

mobilenet_data.index = range(len(mobilenet_data))

fig, axs = plt.subplots(1,3)
plot_data(mobilenet_data, axs[0], "Mobilenet Pruned mem usage per image")
plot_data(squeezenet_data, axs[1], "Squeezenet mem usage per image")
plot_data(efficientnet_data, axs[2], "EfficientNet Pruned mem usage per image")


fig = plt.gcf()
fig.set_size_inches(18,6, forward=True)
plt.show()




In [ ]:
# Graph Max RAM GB vs number of rows, per network, split tick data into high_mem_usage (shown in yellow)
# and not high_mem_usage shown in purple

def plot_data(valid_ram_data, axis, title, color='red'):
    axis.scatter(valid_ram_data['total images'],
            valid_ram_data['Max RAM GB'], 
            c=valid_ram_data['high_mem_usage'],
            #s=valid_ram_data['dataset_x_cols'] * 50,
            alpha=0.5)
    axis.set_title(title)
   


fig, axs = plt.subplots(1,3)
plot_data(mobilenet_data, axs[0], "Mobilenet high vs norm mem usage per image")
# high_mem_mobilenet = mobilenet_data[mobilenet_data['high_mem_usage'] == True]
# axs[0].plot(high_mem_mobilenet['total images'], high_mem_mobilenet['Max RAM GB'])
# draw a line between max and min points
chosen_line = {'x': [7295.0, 281500.0], 'y': [1.333252, 59.120000]}
axs[0].plot(chosen_line['x'], chosen_line['y'])
# so limit equation is y=0.00021074286756259003x−0.20411721886909362
plot_data(squeezenet_data, axs[1], "Squeezenet high vs norm mem usage per image")
plot_data(efficientnet_data, axs[2], "Efficientnet high vs norm mem usage per image")


fig = plt.gcf()
fig.set_size_inches(18,6, forward=True)
plt.show()

In [ ]:
mobilenet_data[mobilenet_data['high_mem_usage'] == True][['total images', 'Max RAM GB', 'dataset_x_cols']].sort_values(by="Max RAM GB", ascending=False)


In [ ]:
# so to explain what it takes to hit the yellow high mem target for Mobilenet, I ran several text models predicting
# `high_mem_usage` and looked at the word cloud...
# I manipulated the blueprint manually to get the word cloud to focus on different pices... removing information
# about the exact dense array used by `LR1` ... i.e. converting C=[<some dense array>] to `C=dense_array_data`
# then the word cloud looks like this:
# dense_array_data, lr1 and poo1 (originally p==1) are all proxy features for using the LR1 model, so that's the memory
# hog
from IPython.display import Image
Image(filename="LR1_word_cloud.jpg")


In [ ]:
# zooming out a bit by including the dense_array_data in the blueprint field then ngrams with highes coef include
# (very small and hard to see):
# batch_size_per_tow ... which is a proxy for KERASMULTIC
# n=1000 ... number of trees I think so signifying a tree based model..
# and to a lesser degree, parameters about where to 

# the wordcloud data shows (similar as above graph(s)) that for mobilenet its the final modeler task code 
# in addition to total_number_images that predicts the memory used.
#
# the task codes that correspond to the much steeper memory use curve are:
# KERASMULTIC, LR1, ESLGBMTC, and SGDC
Image(filename="mobilenet_wordcloud.jpg")

In [ ]:


# various phrases that word cloud showed having high positive coef
# `pp_sf` = stack fit it has coef of 1.0 for high_mem_usage... for image aug task we generally set it to 5
# `batch_size_per_row` = values seem to have a high coef with heavy memory use used in KERASMULTIC
# `t_n=5` == 5 CV folds which indicates more memory usage 
# n=1000   == number of trees I belive, used only in the two tree based models
